© 2020 Nokia

Licensed under the BSD 3 Clause license

SPDX-License-Identifier: BSD-3-Clause

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

import os
import json
import time
import numpy as np
import sys

from codesearch.encoders import BasicEncoder
from codesearch import embedding_pretraining
from codesearch.embedding_pretraining import train_fasttext_model_from_snippets, load_fasttext_model
from codesearch.utils import SaveableFunction
from codesearch.data import load_snippet_collection, EVAL_DATASETS, SNIPPET_COLLECTIONS, eval_datasets_from_regex
from codesearch.ncs.ncs_embedder import TfidfCodeEmbedder, NcsEmbedder
from codesearch.evaluation import evaluate_and_dump 
from codesearch.embedding_retrieval import EmbeddingRetrievalModel
start = time.time()

Read configuration parameters from environment variables (when this notebook is run as a script).

In [2]:
fast_text_checkpoint = os.environ.get("fast_text_checkpoint", None)
model_filename = os.environ.get("model_filename", None)

snippets_collection = os.environ.get("snippets_collection", "conala-curated")
train_snippets_collection = os.environ.get("train_snippets_collection", "conala-curated")
valid_dataset = os.environ.get("valid_dataset", None)
test_dataset = os.environ.get("test_dataset", "conala-curated-0.5-test")

text_overrides = json.loads(os.environ.get("text_overrides", "{}"))
code_overrides = json.loads(os.environ.get("code_overrides", "{}"))
fast_text_overrides = json.loads(os.environ.get("fast_text_overrides", "{}"))
zip_fn_name = os.environ.get("zip_fn", "zip_descr_end")
output_dir = os.environ.get("output_dir", ".")

In [3]:
model_filename, fast_text_checkpoint

(None, None)

In [4]:
snippets_collection

'conala-curated'

In [5]:
text_overrides, code_overrides, fast_text_overrides, zip_fn_name

({}, {}, {}, 'zip_descr_end')

## Load data

In [6]:
if valid_dataset and valid_dataset not in EVAL_DATASETS and valid_dataset not in SNIPPET_COLLECTIONS:
    raise ValueError()
test_datasets = eval_datasets_from_regex(test_dataset)
snippets = load_snippet_collection(snippets_collection)
train_snippets = load_snippet_collection(train_snippets_collection) 

## Train or load embedding model

In [7]:
if fast_text_checkpoint:
    model, enc = load_fasttext_model(fast_text_checkpoint)
    print("Loaded fast text checkpoint")
    
else:
    enc = BasicEncoder(text_preprocessing_params=text_overrides, code_preprocessing_params=code_overrides)
    zip_fn = getattr(sys.modules[embedding_pretraining.__name__], zip_fn_name)
    model = train_fasttext_model_from_snippets(train_snippets, enc, zip_fn, fast_text_overrides, "./", save=False)

Initializing spacy nlp /
Initialized spacy nlp


## Unsupervised retrieval baseline

A first baseline that computes a representation a snippet representation as a tfidf weighted average of their embeddings and a query representation by averaging all terms.

### Embedding code & queries

In [8]:
tfidf_model = TfidfCodeEmbedder.create_tfidf_model(enc, model, snippets)
embedder = NcsEmbedder(model, enc, tfidf_model)

100%|██████████| 2777/2777 [00:00<00:00, 4820.50it/s]


### Create retrieval model

In [9]:
retrieval_model = EmbeddingRetrievalModel(embedder)
retrieval_model.add_snippets(snippets)

Embedding snippets: 100%|██████████| 2777/2777 [00:03<00:00, 774.61it/s]Contains all zero rows



In [10]:
if model_filename: embedder.save(model_filename)

## Evaluation

In [11]:
sample_queries = ["train a tensorflow model", "plot a bar chart", "merge two dataframes", "sort a list", "read a pandas dataframe from a file", "plot an image"]
config = {"text": text_overrides, "code": code_overrides, "fasttext": fast_text_overrides}
evaluate_and_dump(
    retrieval_model, 
    config, 
    output_dir, 
    valid_dataset, 
    test_datasets,
    sample_queries=sample_queries
)

Embedding sequences: 100%|██████████| 1/1 [00:00<00:00, 3833.92it/s]****************************************
QUERY: train a tensorflow model
****************************************

----------------------------------------
RANK 1
----------------------------------------
DESCRIPTION: Filter Django objects by `author` with ids `1` and `2`
CODE: Book.objects.filter(author__id=1).filter(author__id=2)
----------------------------------------

----------------------------------------
RANK 2
----------------------------------------
DESCRIPTION: Update a user's name as `Bob Marley` having id `123` in SQLAlchemy
CODE: session.query(User).filter_by(id=123).update({'name': 'Bob Marley'})
----------------------------------------

----------------------------------------
RANK 3
----------------------------------------
DESCRIPTION: grab one random item from a database `model` in django/postgresql
CODE: model.objects.all().order_by('?')[0]
----------------------------------------

------------------

{'conala-curated-0.5-test': {'mrr': 0.030534542603970444,
  'recall@3': 0.028871391076115485,
  'recall@10': 0.07217847769028872}}

In [12]:
duration = time.time() - start
f"Running the notebook took {duration} seconds"

'Running the notebook took 12.896983623504639 seconds'